In [1]:
import math
import string
import numpy as np

In [2]:
def printPoints(vectors):
    formatted_vectors = []
    for vector in vectors:
        formatted_vector = []
        for value in vector:
            if value == int(value):
                formatted_vector.append(f"{int(value)}")
            else:
                formatted_vector.append(f"{value:.2f}")
        formatted_vectors.append(f"({', '.join(formatted_vector)})")

    alphabet = list(string.ascii_lowercase)
    print('Copy and paste the following directly into Desmos:')
    for char, vector in zip(alphabet, formatted_vectors):
      print(f'O_{char} = {vector}')

In [3]:
def generate_points_on_plane(center, radius, point_a, point_b, num_points):
    """
    Generates a list of orbital waypoints that are centered about the planet's center AND lie on
    the plane of the great circle connecting the planet center, point_a and point_b
    """
    a = 1 * radius

    # Convert points to numpy arrays
    center = np.array(center)
    point_a = np.array(point_a)
    point_b = np.array(point_b)

    # Calculate vectors CA and CB
    vec_ca = point_a - center
    vec_cb = point_b - center

    # Normal vector of the plane (cross product of CA and CB)
    normal = np.cross(vec_ca, vec_cb)

    # Find one basis vector on the plane (we just use CA as basis so that first orbital point is directly above launch)
    u = vec_ca / np.linalg.norm(vec_ca)  # Normalize

    # Find another basis vector on the plane (cross product of normal and U)
    v = np.cross(normal, u)
    v = v / np.linalg.norm(v)  # Normalize

    # Circle's radius
    orbit_radius = radius + a

    # Generate points on the circle
    points = []

    for i in range(num_points):
        theta = 2 * np.pi * i / num_points
        if theta == 0:
            # First point directly above launch site should be slightly lower
            point = center + orbit_radius * 0.8 * (np.cos(theta) * u + np.sin(theta) * v)
        else:
            point = center + orbit_radius * (np.cos(theta) * u + np.sin(theta) * v)

        # EXPERIMENTAL: As we get closer to the target, reduce altitude of orbital point, creating parabola-like descent
        # descent_start_factor = 1.4
        # normalizing_factor = 1 / (descent_start_factor - 1)
        # d = np.linalg.norm(point - point_b)
        # if d <= a * descent_start_factor:
        #     a_d = d - a
        #     a_d_normal = a_d / a
        #     scaling_factor = 1.5 * radius + a * (normalizing_factor * a_d_normal)  # Adjust this formula as needed
        #     point = center + scaling_factor * (np.cos(theta) * u + np.sin(theta) * v)

        points.append(point)

    return np.array(points)


In [4]:
def createPointOnSphere(center, radius, theta, phi):
  h, k, l = center[0], center[1], center[2]
  x_1 = h + radius * math.sin(theta) * math.cos(phi)
  y_1 = k + radius * math.sin(theta) * math.sin(phi)
  z_1 = l + radius * math.cos(theta)
  return (x_1, y_1, z_1)

In [36]:
center = (1, -1, -1)  # Center of the sphere
radius = 5  # Radius of the sphere
point_a = createPointOnSphere(center, radius, 2.01, 0)
point_b = createPointOnSphere(center, radius + 8, 1.18, 2.6)
num_points = 24  # Number of points to generate

# Generate the points
points_on_plane = generate_points_on_plane(center, radius, point_a, point_b, num_points)
printPoints(points_on_plane)
print(point_b)

Copy and paste the following directly into Desmos:
O_a = (8.24, -1, -4.40)
O_b = (9.76, 1.59, -5.07)
O_c = (8.87, 4.00, -4.61)
O_d = (7.45, 6.07, -3.90)
O_e = (5.59, 7.66, -3.00)
O_f = (3.41, 8.66, -1.96)
O_g = (1.07, 9.00, -0.85)
O_h = (-1.27, 8.66, 0.25)
O_i = (-3.46, 7.66, 1.26)
O_j = (-5.35, 6.07, 2.11)
O_k = (-6.80, 4.00, 2.76)
O_l = (-7.72, 1.59, 3.15)
O_m = (-8.05, -1.00, 3.25)
O_n = (-7.76, -3.59, 3.07)
O_o = (-6.87, -6.00, 2.61)
O_p = (-5.45, -8.07, 1.90)
O_q = (-3.59, -9.66, 1.00)
O_r = (-1.41, -10.66, -0.04)
O_s = (0.93, -11.00, -1.15)
O_t = (3.27, -10.66, -2.25)
O_u = (5.46, -9.66, -3.26)
O_v = (7.35, -8.07, -4.11)
O_w = (8.80, -6.00, -4.76)
O_x = (9.72, -3.59, -5.15)
(-9.299698413286645, 5.1962636812792065, 3.9520227167694637)


In [37]:
def pointIsEqualToEither(point, target_one, target_two):
  return np.array_equal(point, target_one) or np.array_equal(point, target_two)

: 

In [28]:
def findShortestPath(path_a, path_b, target_one, target_two):
  for ind in range(0, len(path_a)):
    if pointIsEqualToEither(path_a[ind], target_one, target_two):
      return path_a[:ind + 1]
    if pointIsEqualToEither(path_b[ind], target_one, target_two):
      return path_b[:ind + 1]

In [29]:
# point_b should be the target point
def findShortestFlightPath(target_point, points_on_plane):
  # Get each point's distance from the target point and store as a tuple (point, distanceFromTarget)
  orbital_point_distances_from_target = []
  for point in points_on_plane:
    orbital_point_distances_from_target.append((point, np.linalg.norm(target_point - point)))

  # Sort keys from shortest distance to the target point to the furthest distance
  sorted_distances = sorted(orbital_point_distances_from_target, key=lambda x: x[1])

  # the points themselves
  closest_point_to_target = sorted_distances[0][0]
  second_closest_point_to_target = sorted_distances[1][0]
  third_closest_point_to_target = sorted_distances[2][0]

  # the points' distances from the target
  second_closest_point_distance = sorted_distances[1][1]
  third_closest_point_distance = sorted_distances[2][1]

  path_a = points_on_plane
  # create another list of points where the first point is the same but all others are reversed
  path_b = points_on_plane[:1] + points_on_plane[-1:0:-1]

  # Edge case: target point is almost directly below an orbital point, meaning the second and third closest points are equidistant from the target point
  if abs(second_closest_point_distance - third_closest_point_distance) < 1e-6:
    return findShortestPath(path_a, path_b, second_closest_point_to_target, third_closest_point_to_target)

  return findShortestPath(path_a, path_b, closest_point_to_target, second_closest_point_to_target)

In [34]:
def findEarlyBreakoffOrbit(orbit, target_point):
  if len(orbit) <= 2:
      return orbit

  # Temporarily put the launch coordinates into the orbit list
  orbit = np.insert(orbit, 0, point_a, axis=0)

  angles = np.zeros(len(orbit))

  for i in range(len(orbit) - 1):
      currentPoint = orbit[i]
      nextPoint = orbit[i + 1]
      angles[i] = findAngleBetween(currentPoint, nextPoint, target_point)

  # Find the index of the angle closest to pi
  min_difference = np.abs(angles - np.pi)
  optimalExitPointIndex = np.argmin(min_difference) + 1

  # Start from the second index to effectively remove the launch coordinates from the orbit list
  # Add one to optimalExitPointIndex to ensure we are inclusive of the optimal exit point
  return orbit[1:optimalExitPointIndex + 1]

def findAngleBetween(currentPoint, nextPoint, targetPoint):
  NC = currentPoint - nextPoint
  NT = targetPoint - nextPoint
  dotProduct = np.dot(NC, NT)
  magnitudes = np.linalg.norm(NC) * np.linalg.norm(NT)
  angle = math.acos(dotProduct / magnitudes)
  return angle

In [35]:
printPoints(findEarlyBreakoffOrbit(findShortestFlightPath(point_b, points_on_plane), point_b))

Copy and paste the following directly into Desmos:
O_a = (8.24, -1, -4.40)
O_b = (9.76, 1.59, -5.07)
O_c = (8.87, 4.00, -4.61)
O_d = (7.45, 6.07, -3.90)
O_e = (5.59, 7.66, -3.00)
O_f = (3.41, 8.66, -1.96)
O_g = (1.07, 9.00, -0.85)
O_h = (-1.27, 8.66, 0.25)
